In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from autogluon.tabular import TabularPredictor
from sklearn.model_selection._split import _BaseKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


## Import the datasets

In [2]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

x_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
x_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
x_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

x_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
x_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
x_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

x_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
x_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
x_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

## Merge x_train observed and x_train_estimated

In [3]:
x_train_a = pd.concat([x_train_observed_a,x_train_estimated_a])
x_train_b = pd.concat([x_train_observed_b,x_train_estimated_b])
x_train_c = pd.concat([x_train_observed_c,x_train_estimated_c])

In [4]:
x_test_estimated_a

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-04-30 07:00:04,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,0.0,0.0,0.0,0.0,80.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.0,0.0,1000.700012,1007.000000,0.0,0.0,80.300003,1013.299988,NaN,0.0,0.0,0.0,0.0,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-04-30 07:00:04,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,0.0,0.0,0.0,0.0,77.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.700012,0.0,0.0,1000.599976,1006.900024,0.0,0.0,80.300003,1013.200012,NaN,0.0,0.0,0.0,0.0,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-04-30 07:00:04,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,0.0,0.0,0.0,0.0,73.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.599976,0.0,0.0,1000.500000,1006.700012,0.0,0.0,80.300003,1013.000000,NaN,0.0,0.0,0.0,0.0,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-04-30 07:00:04,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,0.0,0.0,0.0,0.0,69.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.500000,0.0,0.0,1000.400024,1006.599976,0.0,0.0,80.199997,1012.900024,NaN,0.0,0.0,0.0,0.0,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-04-30 07:00:04,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,0.0,0.0,0.0,0.0,64.500000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.299988,0.0,0.0,1000.200012,1006.500000,0.0,0.0,80.199997,1012.799988,NaN,0.0,0.0,0.0,0.0,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-07-02 07:00:31,2023-07-03 22:45:00,9.1,1.207,2090.600098,0.0,0.0,1629.400024,0.0,282.700012,0.0,0.0,0.0,0.0,99.300003,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.299988,0.0,0.0,980.799988,986.700012,0.0,0.0,82.199997,992.599976,NaN,0.0,0.0,-0.0,0.0,351.855988,-3.282,0.1,285.399994,99.400002,39931.601562,2.1,1.7,-1.3,0.0
2876,2023-07-02 07:00:31,2023-07-03 23:00:00,9.1,1.207,2015.800049,0.0,0.0,1537.300049,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.400024,0.0,0.0,980.900024,986.799988,0.0,0.0,82.500000,992.700012,NaN,0.0,0.0,-0.0,0.0,355.317993,-3.469,0.1,285.299988,100.000000,39574.300781,2.0,1.6,-1.3,0.0
2877,2023-07-02 07:00:31,2023-07-03 23:15:00,9.1,1.208,2015.699951,0.0,0.0,1471.400024,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.500000,0.0,0.0,981.000000,986.900024,0.0,0.0,82.800003,992.799988,NaN,0.0,0.0,-0.0,0.0,358.785004,-3.556,0.1,285.299988,100.000000,39627.199219,2.0,1.6,-1.2,0.0
2878,2023-07-02 07:00:31,2023-07-03 23:30:00,9.0,1.208,2015.599976,0.0,0.0,1405.500000,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.599976,0.0,0.0,981.000000,986.900024,0.0,0.0,83.199997,992.900024,NaN,0.0,0.0,-0.0,0.0,2.253000,-3.541,0.1,285.200012,100.000000,39691.800781,2.0,1.6,-1.1,0.0


## Part 1: Data Preprocessing

1. Aggregate data from every 15 minutes to hourly intervals:
   - Method 1: Take the mean over all four 15minutes recording, resulting in hourly measurements.
   - Method 2: Create a separate column for each 15-minute value, effectively quadrupling the number of columns.
2. Handle missing values
   - Remove rows with NaN values in pv_measurement
   - Remove rows with timestamps that are not present in both x and y
3. 


### 1. Transform data to hourly

**Method 1: Take the mean over all four 15minutes recording, resulting in hourly measurements.**

In [5]:
def resample_to_hourly(x): 
    df = x.copy()
    df.set_index('date_forecast', inplace=True)
    
    # Aggregating by averaging over quartarly measurements
    df_hourly = df.resample('H').mean(numeric_only=False)
    df_hourly.reset_index(inplace=True)
    return df_hourly

x_train_a_hourly = resample_to_hourly(x_train_a)
x_train_b_hourly = resample_to_hourly(x_train_b)
x_train_c_hourly = resample_to_hourly(x_train_c)

x_test_a_hourly = resample_to_hourly(x_test_estimated_a)
x_test_b_hourly = resample_to_hourly(x_test_estimated_b)
x_test_c_hourly = resample_to_hourly(x_test_estimated_c)

In [6]:
#use only rows in test that are given in the test csv
test = pd.read_csv('test.csv')
pred_time_stamps = test['time'].unique()
x_test_a_hourly = x_test_a_hourly[x_test_a_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_b_hourly = x_test_b_hourly[x_test_b_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_c_hourly = x_test_c_hourly[x_test_c_hourly['date_forecast'].isin(pred_time_stamps)]

**Method 2: Create a separate column for each 15-minute value, quadrupling the number of columns.**

In [7]:
def resample_to_hourly_quarters(x, date_column='date_forecast', exclude_column='date_calc'):
    df = x.copy()
    # Ensure the date column is in datetime format and set as the index
    df[date_column] = pd.to_datetime(df[date_column])
    df.set_index(date_column, inplace=True)
    
    # Separate the column to exclude from the resampling
    excluded_data = df[[exclude_column]].resample('H').first()  # You can use 'first' or 'last' here
    
    # Remove the excluded column from df before pivoting
    df = df.drop(columns=[exclude_column])

    # Add a column for the 15-minute period within the hour
    df['quarter'] = df.index.minute // 15  # Use floor division to get the quarter number (0, 1, 2, 3)

    # Pivot the table. For each feature, create a new column for each 15-minute period.
    df_pivot = df.pivot_table(index=df.index.floor('H'),
                              columns='quarter',
                              aggfunc='first')  # We use 'first' because each quarter should be unique
    
    # Flatten the multi-level column index
    df_pivot.columns = ['{}_Q{}'.format(feature, quarter) for feature, quarter in df_pivot.columns]

    # Reset the index to be able to merge on the date_column
    df_pivot.reset_index(inplace=True)
    excluded_data.reset_index(inplace=True)

    # Merge back the excluded column
    df_hourly = pd.merge(excluded_data, df_pivot, on=date_column)

    return df_hourly

# Make sure to pass the column name that contains the datetime information
x_train_a_hourly2 = resample_to_hourly_quarters(x_train_a, date_column='date_forecast')
x_train_b_hourly2 = resample_to_hourly_quarters(x_train_b, date_column='date_forecast')
x_train_c_hourly2 = resample_to_hourly_quarters(x_train_c, date_column='date_forecast')

x_test_a_hourly2 = resample_to_hourly_quarters(x_test_estimated_a, date_column='date_forecast')
x_test_b_hourly2 = resample_to_hourly_quarters(x_test_estimated_b, date_column='date_forecast')
x_test_c_hourly2 = resample_to_hourly_quarters(x_test_estimated_c, date_column='date_forecast')

## 2. Handle consecutive pv measurments

In [8]:
#Filters out rows from a DataFrame where the 'pv_measurement' column has consecutive identical values beyond a specified threshold.
def remove_constant_intervals(y_train, low_thresh, upp_thresh = 10**6):
    """
    Identify and remove intervals of constant PV readings that exceed a specified duration. 
    Constant readings may indicate sensor malfunctions or data logging issues.
    """
    
    df = y_train.copy()
    
    # Calculate the difference in production values
    df['diff'] = df['pv_measurement'].diff()

    # Identify where the difference is zero
    df['zero_diff'] = df['diff'].abs() < 1e-5

    # Identify groups of consecutive zero differences
    df['group'] = (df['zero_diff'] != df['zero_diff'].shift()).cumsum()

    # Filter out only the groups with consecutive zero differences
    constant_intervals = df[df['zero_diff']].groupby('group').agg(start=('time', 'min'), 
                                                                  end=('time', 'max'),
                                                                  duration=('time', 'size'))
    
    # Filter intervals based on the threshold
    interval_df_thresh = constant_intervals[(constant_intervals['duration'] > low_thresh) & (constant_intervals['duration'] <upp_thresh)]
    
    # Remove rows from the main dataframe that fall within these intervals
    for _, row in interval_df_thresh.iterrows():
        start_time, end_time = row['start'], row['end']
        df = df[(df['time'] < start_time) | (df['time'] > end_time)]
    
    # Drop the added columns used for calculations
    df.drop(columns=['diff', 'zero_diff', 'group'], inplace=True)
    
    return df

In [9]:
#Remove rows in groups of constant values, where duration of constant measurements is > 1 day (24 hours)
train_a = remove_constant_intervals(train_a,24)
train_b = remove_constant_intervals(train_b,24)
train_c = remove_constant_intervals(train_c,24)

## 3.Handle missing values and rows 
   - Remove NaN pv measurement values from train a/b/c
   - Remove rows that have timestamps that are not present in both x and y

In [10]:
"""
# Identify the indices of the rows with NaN values in the 'pv_measurement' column
nan_indices_a = train_a[train_a['pv_measurement'].isna()].index
nan_indices_b = train_b[train_b['pv_measurement'].isna()].index
nan_indices_c = train_c[train_c['pv_measurement'].isna()].index

# Drop these indices from y_train
train_a = train_a.drop(nan_indices_a).reset_index(drop = True)
train_b = train_b.drop(nan_indices_b).reset_index(drop = True)
train_c = train_c.drop(nan_indices_c).reset_index(drop = True)

"""

"\n# Identify the indices of the rows with NaN values in the 'pv_measurement' column\nnan_indices_a = train_a[train_a['pv_measurement'].isna()].index\nnan_indices_b = train_b[train_b['pv_measurement'].isna()].index\nnan_indices_c = train_c[train_c['pv_measurement'].isna()].index\n\n# Drop these indices from y_train\ntrain_a = train_a.drop(nan_indices_a).reset_index(drop = True)\ntrain_b = train_b.drop(nan_indices_b).reset_index(drop = True)\ntrain_c = train_c.drop(nan_indices_c).reset_index(drop = True)\n\n"

In [11]:
# Remove all rows with date-time values that are not present in both x and y in order to synchronize x and its labels. 
def remove_non_synchronous_rows(x_train, y_train, x_date_column='date_forecast', y_date_column='time'):
    # Convert date columns to datetime format for easier comparison
    x_train[x_date_column] = pd.to_datetime(x_train[x_date_column])
    y_train[y_date_column] = pd.to_datetime(y_train[y_date_column])
    
    # Find common dates
    common_dates = x_train[x_date_column][x_train[x_date_column].isin(y_train[y_date_column])]
    
    # Filter both datasets based on common dates
    x_train_synced = x_train.loc[x_train[x_date_column].isin(common_dates)]
    y_train_synced = y_train.loc[y_train[y_date_column].isin(common_dates)]
    
    return x_train_synced, y_train_synced

# Remove the rows with date and time that only shows up in one of the sets
x_train_a_hourly, train_a = remove_non_synchronous_rows(x_train_a_hourly, train_a)
x_train_b_hourly, train_b = remove_non_synchronous_rows(x_train_b_hourly, train_b)
x_train_c_hourly, train_c = remove_non_synchronous_rows(x_train_c_hourly, train_c)

# Remove the rows with date and time that only shows up in one of the sets
x_train_a_hourly2, train_a = remove_non_synchronous_rows(x_train_a_hourly2, train_a)
x_train_b_hourly2, train_b = remove_non_synchronous_rows(x_train_b_hourly2, train_b)
x_train_c_hourly2, train_c = remove_non_synchronous_rows(x_train_c_hourly2, train_c)

In [12]:
#Remove snow:density column as well as rows with only NaN values
def remove_nan_rows_cols(x,remove_cols):
    df = x.copy()
    
    nan_cols = ['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms']
    
    nan_rows_mask = df.loc[:,nan_cols].isna().all(axis=1)
    df = df.drop(df[nan_rows_mask].index, inplace=False)
    
    df = df.drop(columns = remove_cols ) #Should we include 'cloud_base_agl:m' and ceiling_height_agl:m ['snow_density:kgm3']
    return df



In [13]:
x_train_a_hourly = remove_nan_rows_cols(x_train_a_hourly,['snow_density:kgm3'])
x_train_b_hourly = remove_nan_rows_cols(x_train_b_hourly,['snow_density:kgm3'])
x_train_c_hourly = remove_nan_rows_cols(x_train_c_hourly,['snow_density:kgm3'])

x_test_a_hourly = remove_nan_rows_cols(x_test_a_hourly,['snow_density:kgm3'])
x_test_b_hourly = remove_nan_rows_cols(x_test_b_hourly,['snow_density:kgm3'])
x_test_c_hourly = remove_nan_rows_cols(x_test_c_hourly,['snow_density:kgm3'])

"""
x_train_a_hourly2 =  remove_nan_rows_cols(x_train_a_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])
x_train_b_hourly2 =  remove_nan_rows_cols(x_train_b_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])
x_train_c_hourly2 =  remove_nan_rows_cols(x_train_c_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])

x_test_a_hourly2 =  remove_nan_rows_cols(x_test_a_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])
x_test_b_hourly2 =  remove_nan_rows_cols(x_test_b_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])
x_test_c_hourly2 =  remove_nan_rows_cols(x_test_x_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])
"""

"\nx_train_a_hourly2 =  remove_nan_rows_cols(x_train_a_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])\nx_train_b_hourly2 =  remove_nan_rows_cols(x_train_b_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])\nx_train_c_hourly2 =  remove_nan_rows_cols(x_train_c_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])\n\nx_test_a_hourly2 =  remove_nan_rows_cols(x_test_a_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])\nx_test_b_hourly2 =  remove_nan_rows_cols(x_test_b_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])\nx_test_c_hourly2 =  remove_nan_rows_cols(x_test_x_hourly2,['snow_density:kgm3', 'cloud_base_agl:m'])\n"

### 3. Feature Engineering 
   - Add time features: hour, day, month, year
   - Add binary observed column
   - Add cyclical features
   - Add direct_rad x sun_elevation feature

In [14]:
# Extracts year, month, day, and hour features from a given datetime column
def extract_date_features(X):
    df = X.copy()
    # Extract features
    df['year'] = df['date_forecast'].dt.year.astype(int)
    df['month'] = df['date_forecast'].dt.month.astype(int)
    df['day'] = df['date_forecast'].dt.day.astype(int)
    df['hour'] = df['date_forecast'].dt.hour.astype(int)
    
    df['observed'] = (df['date_calc'].isna()).astype(int)
    df['observed'] = df['observed'].astype(str)
    
    
    df = df.drop(columns = ['date_calc'])
    
    return df

In [15]:
x_train_a = extract_date_features(x_train_a_hourly)
x_train_b = extract_date_features(x_train_b_hourly)
x_train_c = extract_date_features(x_train_c_hourly)

x_test_a = extract_date_features(x_test_a_hourly)
x_test_b = extract_date_features(x_test_b_hourly)
x_test_c = extract_date_features(x_test_c_hourly)

In [16]:
x_train_a2 = extract_date_features(x_train_a_hourly2)
x_train_b2 = extract_date_features(x_train_b_hourly2)
x_train_c2 = extract_date_features(x_train_c_hourly2)

x_test_a2 = extract_date_features(x_test_a_hourly2)
x_test_b2 = extract_date_features(x_test_b_hourly2)
x_test_c2 = extract_date_features(x_test_c_hourly2)

In [20]:
# Creating cyclical features for hour of the day and month of the year
def add_cyclic(x_train):
    train_data = x_train.copy()
   
    train_data['hour_sin'] = np.sin(2 * np.pi * train_data['hour'] / 24)
    train_data['hour_cos'] = np.cos(2 * np.pi * train_data['hour'] / 24)
    train_data['month_sin'] = np.sin(2 * np.pi * (train_data['month']-1) / 12)
    train_data['month_cos'] = np.cos(2 * np.pi * (train_data['month']-1) / 12)
    
    #train_data.drop(columns = ['hour','month'],inplace = True)
    return train_data

x_train_a = add_cyclic(x_train_a)
x_train_b = add_cyclic(x_train_b)
x_train_c = add_cyclic(x_train_c)

x_test_a = add_cyclic(x_test_a)
x_test_b = add_cyclic(x_test_b)
x_test_c = add_cyclic(x_test_c)

x_train_a2 = add_cyclic(x_train_a2)
x_train_b2 = add_cyclic(x_train_b2)
x_train_c2 = add_cyclic(x_train_c2)

x_test_a2 = add_cyclic(x_test_a2)
x_test_b2 = add_cyclic(x_test_b2)
x_test_c2 = add_cyclic(x_test_c2)

## Part 2: Model Building 

### 1. Catboost

In [21]:
merged_a = pd.merge(x_train_a, train_a, left_on='date_forecast', right_on='time', how='inner')
merged_b = pd.merge(x_train_b, train_b, left_on='date_forecast', right_on='time', how='inner')
merged_c = pd.merge(x_train_c, train_c, left_on='date_forecast', right_on='time', how='inner')

In [27]:
train_a

,time,pv_measurement
0,2019-06-02 22:00:00,0.00
1,2019-06-02 23:00:00,0.00
2,2019-06-03 00:00:00,0.00
3,2019-06-03 01:00:00,0.00
4,2019-06-03 02:00:00,19.36
...,...,...
34080,2023-04-30 19:00:00,9.02
34081,2023-04-30 20:00:00,0.00
34082,2023-04-30 21:00:00,0.00
34083,2023-04-30 22:00:00,0.00


In [24]:
merged_a

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,year,month,day,hour,observed,hour_sin,hour_cos,month_sin,month_cos,time,pv_measurement
0,2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000,0.000,1728.949951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1006.299988,0.0,0.0,993.750000,999.775024,0.0,0.000,71.674995,1005.799988,0.0,0.0,0.0,0.175,348.036743,-3.77425,0.000,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,2019,6,2,22,1,-0.500000,0.866025,0.5,-8.660254e-01,2019-06-02 22:00:00,0.00
1,2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000,0.000,1689.824951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1005.200012,0.0,0.0,992.674988,998.650024,0.0,0.025,68.000000,1004.650024,0.0,0.0,0.0,0.200,91.980751,-4.35725,0.000,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,2019,6,2,23,1,-0.258819,0.965926,0.5,-8.660254e-01,2019-06-02 23:00:00,0.00
2,2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000,0.000,1563.224976,0.0,280.649994,0.000,0.000000,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1004.525024,0.0,0.0,992.000000,997.974976,0.0,0.100,67.949997,1003.950012,0.0,0.0,0.0,0.400,14.934750,-3.30950,0.000,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,2019,6,3,0,1,0.000000,1.000000,0.5,-8.660254e-01,2019-06-03 00:00:00,0.00
3,2019-06-03 01:00:00,8.425,1.21800,1283.425049,208.649994,0.750,1283.425049,0.0,281.674988,0.300,526.775024,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.25,1.00,1004.025024,0.0,0.0,991.500000,997.449951,0.0,0.125,73.875000,1003.449951,0.0,0.0,0.0,0.550,28.630251,-0.82250,0.000,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,2019,6,3,1,1,0.258819,0.965926,0.5,-8.660254e-01,2019-06-03 01:00:00,0.00
4,2019-06-03 02:00:00,8.950,1.21800,1003.500000,32468.150391,23.100,1003.500000,0.0,282.500000,11.975,22068.949219,0.15,282.975006,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1003.099976,0.0,0.0,990.550049,996.500000,0.0,0.100,79.925003,1002.500000,0.0,0.0,0.0,0.250,41.997501,3.05125,0.000,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,2019,6,3,2,1,0.500000,0.866025,0.5,-8.660254e-01,2019-06-03 02:00:00,19.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34014,2023-04-30 19:00:00,4.550,1.27650,1674.199951,84464.945312,4.225,542.700012,0.0,272.424988,2.825,56431.050781,0.10,13230.649414,96.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.50,0.75,1014.750000,0.0,0.0,1001.574951,1007.799988,0.0,0.000,74.625000,1014.049988,0.0,0.0,0.0,0.000,304.939240,-0.18050,0.000,275.024994,96.699997,23417.074219,5.175,4.800,1.925,0.0,2023,4,30,19,0,-0.965926,0.258819,1.0,6.123234e-17,2023-04-30 19:00:00,9.02
34015,2023-04-30 20:00:00,4.500,1.27975,1762.400024,2270.875000,0.000,546.400024,0.0,272.299988,0.000,5067.875000,0.00,176.350006,94.224998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1014.700012,0.0,0.0,1001.549988,1007.799988,0.0,0.000,76.875000,1014.049988,0.0,0.0,0.0,0.000,318.625763,-5.17600,0.000,274.650024,94.525002,21084.050781,4.650,4.025,2.300,0.0,2023,4,30,20,0,-0.866025,0.50000

In [25]:
merged_b

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,year,month,day,hour,observed,hour_sin,hour_cos,month_sin,month_cos,time,pv_measurement
0,2019-01-01 00:00:00,5.525,1.23975,1200.675049,0.000000,0.000,1200.675049,0.0,275.149994,0.000,0.000000,0.0,0.000000,81.824997,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,987.000000,0.0375,0.75,974.099976,980.174988,0.0,0.0,89.099998,986.224976,0.0,0.0,0.0,0.050,20.724251,-48.319252,0.000,278.250000,86.324997,16827.800781,2.800,1.175,2.550,0.0,2019,1,1,0,1,0.000000,1.000000,0.0,1.000000e+00,2019-01-01 00:00:00,0.000000
1,2019-01-01 01:00:00,5.425,1.23975,1131.424927,0.000000,0.000,1131.424927,0.0,274.825012,0.000,0.000000,0.0,0.000000,97.474998,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,986.150024,0.0125,0.25,973.250000,979.325012,0.0,0.0,88.425003,985.400024,0.0,0.0,0.0,0.100,40.307251,-44.919750,0.000,278.075012,99.074997,12777.449219,3.550,1.525,3.200,0.0,2019,1,1,1,1,0.258819,0.965926,0.0,1.000000e+00,2019-01-01 01:00:00,0.000000
2,2019-01-01 02:00:00,5.400,1.23850,1061.000000,0.000000,0.000,1061.000000,0.0,274.799988,0.000,0.000000,0.0,0.000000,96.425003,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,985.950012,0.0000,0.00,973.049988,979.099976,0.0,0.0,87.300003,985.174988,0.0,0.0,0.0,0.100,57.805000,-39.863998,0.100,278.049988,97.925003,11140.500000,4.025,2.150,3.425,0.0,2019,1,1,2,1,0.500000,0.866025,0.0,1.000000e+00,2019-01-01 02:00:00,0.000000
3,2019-01-01 03:00:00,5.350,1.23975,1021.150024,0.000000,0.000,1021.150024,0.0,274.674988,0.000,0.000000,0.0,0.000000,96.775002,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,986.250000,0.0450,0.75,973.349976,979.349976,0.0,0.0,88.400002,985.450012,0.0,0.0,0.0,0.100,73.371002,-33.769001,0.200,277.600006,97.224998,6799.200195,4.900,3.425,3.500,0.0,2019,1,1,3,1,0.707107,0.707107,0.0,1.000000e+00,2019-01-01 03:00:00,0.000000
4,2019-01-01 04:00:00,5.675,1.23750,1033.699951,0.000000,0.000,1033.699951,0.0,275.500000,0.000,0.000000,0.0,0.000000,96.724998,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,986.775024,0.0150,0.25,973.875000,979.900024,0.0,0.0,91.275002,985.974976,0.0,0.0,0.0,0.100,87.514999,-27.167500,0.175,277.649994,96.824997,5580.475098,6.050,5.575,2.200,0.0,2019,1,1,4,1,0.866025,0.500000,0.0,1.000000e+00,2019-01-01 04:00:00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25953,2023-04-30 19:00:00,4.550,1.27650,1677.949951,84462.523438,4.225,542.849976,0.0,272.424988,2.825,56433.972656,0.1,13244.500000,96.724998,7.0,0.0,0.0,0.0,0.0,0.0,0.5,0.75,1014.750000,0.0000,0.00,1001.450012,1007.674988,0.0,0.0,74.625000,1013.950012,0.0,0.0,0.0,0.000,304.940247,-0.180500,0.000,275.024994,96.724998,23434.074219,5.175,4.800,1.925,0.0,2023,4,30,19,0,-0.965926,0.258819,1.0,6.123234e-17,2023-04-30 19:00:00,0.828587
25954,2023-04-30 20:00:00,4.500,1.27875,1766.500000,2270.774902,0.000,546.349976,0.0,272.299988,0.000,5067.024902,0.0,176.649994,94.224998,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,1014.750000,0.0000,0.00,1001.450012,1007.674988,0.0,0.0,76.849998,1013.950012,0.0,0.0,0.0,0.000,318.626251,-5.175750,0.000,274.650024,94.525002,21307.724609,4.650,4.025,2.300,0.0,2023,4,30,20,0,-0.866025,0.500000,1.0,6.12323

In [26]:
merged_c

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,year,month,day,hour,observed,hour_sin,hour_cos,month_sin,month_cos,time,pv_measurement
0,2019-01-01 00:00:00,5.475,1.24075,1130.025024,0.000000,0.000,1130.025024,1.00,274.975006,0.000,0.000000,0.0,0.000000,78.699997,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.949951,0.045,0.75,972.150024,978.174988,0.20,0.0,89.949997,984.250000,0.0,0.0,0.0,0.050,20.734001,-48.355999,0.000,278.350006,83.824997,16509.250000,2.900,1.125,2.625,0.0,2019,1,1,0,1,0.000000,1.000000,0.0,1.000000e+00,2019-01-01 00:00:00,NaN
1,2019-01-01 01:00:00,5.350,1.24000,1066.674927,0.000000,0.000,1066.674927,0.75,274.649994,0.000,0.000000,0.0,0.000000,97.375000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.174988,0.015,0.25,971.250000,977.299988,0.15,0.0,89.000000,983.375000,0.0,0.0,0.0,0.100,40.327751,-44.950500,0.000,278.200012,99.074997,13277.525391,3.650,1.425,3.350,0.0,2019,1,1,1,1,0.258819,0.965926,0.0,1.000000e+00,2019-01-01 01:00:00,NaN
2,2019-01-01 02:00:00,5.350,1.23900,1002.349976,0.000000,0.000,1002.349976,0.00,274.649994,0.000,0.000000,0.0,0.000000,95.900002,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,985.974976,0.000,0.00,971.075012,977.099976,0.00,0.0,87.699997,983.174988,0.0,0.0,0.0,0.100,57.828251,-39.885750,0.100,278.149994,97.650002,11213.525391,3.950,1.900,3.475,0.0,2019,1,1,2,1,0.500000,0.866025,0.0,1.000000e+00,2019-01-01 02:00:00,NaN
3,2019-01-01 03:00:00,5.300,1.24000,959.150024,0.000000,0.000,959.150024,0.00,274.475006,0.000,0.000000,0.0,0.000000,96.074997,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.299988,0.045,0.75,971.349976,977.450012,0.00,0.0,88.949997,983.549988,0.0,0.0,0.0,0.075,73.391998,-33.781502,0.200,277.625000,96.625000,7082.125000,4.600,3.025,3.400,0.0,2019,1,1,3,1,0.707107,0.707107,0.0,1.000000e+00,2019-01-01 03:00:00,NaN
4,2019-01-01 04:00:00,5.625,1.23825,952.349976,0.000000,0.000,952.349976,0.00,275.299988,0.000,0.000000,0.0,0.000000,96.925003,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.900024,0.015,0.25,971.900024,978.000000,0.00,0.0,92.750000,984.025024,0.0,0.0,0.0,0.100,87.531754,-27.170500,0.200,277.500000,96.974998,5347.424805,5.425,4.925,2.175,0.0,2019,1,1,4,1,0.866025,0.500000,0.0,1.000000e+00,2019-01-01 04:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27199,2023-04-30 19:00:00,4.400,1.27550,1456.574951,84010.148438,4.175,551.224976,0.00,272.024994,2.775,54774.000000,0.0,9934.575195,97.724998,24.0,0.0,0.0,0.3,0.0,0.0,0.5,1.0,1014.900024,0.000,0.00,999.049988,1005.275024,0.00,0.0,74.800003,1011.525024,0.0,0.0,0.0,0.000,304.936493,-0.201500,0.000,274.924988,97.724998,25028.000000,4.075,3.600,1.875,0.0,2023,4,30,19,0,-0.965926,0.258819,1.0,6.123234e-17,2023-04-30 19:00:00,50.96
27200,2023-04-30 20:00:00,4.400,1.27850,1476.349976,2206.800049,0.000,564.099976,0.00,271.950012,0.000,4984.049805,0.0,0.000000,95.449997,24.0,0.0,0.0,0.3,0.0,0.0,0.0,1.0,1014.849976,0.000,0.00,999.025024,1005.250000,0.00,0.0,76.974998,1011.549988,0.0,0.0,0.0,0.000,318.620483,-5.204000,0.000,274.575012,95.850006,23995.599609,3.600,2.950,2.125,0.0,2023,4,30,20,0,-0.866025,0.500000,1.0,6.123234e-17,2023-04-30 20:00:00,2.94

In [22]:
def build_catboost_multiple_seed(merged_df,x_test,number_of_models):
    merged_df = merged_df.drop(columns=['date_forecast', 'time'])
    X = merged_df.drop(columns=['pv_measurement'])
    y = merged_df['pv_measurement']
    
    predictions = []
    models = []
    scores = []
    seeds = range(number_of_models)
    
    for seed in seeds:
        X_train, X_validation, y_train, y_validation = train_test_split(
            X, y, train_size=0.8, random_state=seed)
        
        catboost_model = CatBoostRegressor(
            cat_features=['observed'],
            iterations=10000,
            learning_rate=0.1,
            depth=6,
            loss_function='MAE',
            eval_metric='MAE',
            random_seed=seed,
            verbose=200
        )
        
        catboost_model.fit(X_train, y_train, eval_set=(X_validation, y_validation),
                           use_best_model=True, early_stopping_rounds=200)
        
        score = catboost_model.get_best_score()['validation']['MAE']
        scores.append(score)
        # Print the best validation MAE for the current seed
        print(f"Best validation MAE for seed {seed}: {score}")
        
        
        # Predict using the current model
        preds = catboost_model.predict(x_test)
        predictions.append(preds)
        models.append(catboost_model)
    
    # Average the predictions from all models
    averaged_predictions = np.mean(predictions, axis=0)
    average_score = np.mean(scores, axis = 0)
    
    return averaged_predictions,models, average_score

In [23]:
pred_a, models_a, avg_a = build_catboost_multiple_seed(merged_a,x_test_a,10)
pred_b, models_b, avg_b = build_catboost_multiple_seed(merged_b,x_test_b,10)
pred_c, models_c, avg_c= build_catboost_multiple_seed(merged_c,x_test_c,10)

0:	learn: 583.7446175	test: 601.9980832	best: 601.9980832 (0)	total: 87.6ms	remaining: 14m 36s
200:	learn: 185.8090543	test: 194.9324312	best: 194.9324312 (200)	total: 2.96s	remaining: 2m 24s
400:	learn: 171.4713252	test: 187.4856701	best: 187.4856701 (400)	total: 5.69s	remaining: 2m 16s
600:	learn: 163.7304040	test: 184.3851517	best: 184.3850770 (599)	total: 7.86s	remaining: 2m 2s
800:	learn: 158.2052404	test: 182.3577627	best: 182.3577627 (800)	total: 10.1s	remaining: 1m 55s
1000:	learn: 153.5286275	test: 181.3651376	best: 181.3651376 (1000)	total: 12.2s	remaining: 1m 49s
1200:	learn: 149.8737580	test: 180.2530526	best: 180.2530526 (1200)	total: 14.2s	remaining: 1m 44s
1400:	learn: 145.7199321	test: 179.3874434	best: 179.3856318 (1385)	total: 16.5s	remaining: 1m 41s
1600:	learn: 142.6327839	test: 178.7861887	best: 178.7835656 (1584)	total: 18.6s	remaining: 1m 37s
1800:	learn: 139.9537399	test: 178.1622931	best: 178.1616605 (1793)	total: 20.9s	remaining: 1m 34s
2000:	learn: 137.490557

CatBoostError: /src/catboost/catboost/libs/data/model_dataset_compatibility.cpp:53: Feature hour is Float in model but marked different in the dataset

In [ ]:
x_test_a

In [ ]:
print(avg_a, avg_b, avg_c)

In [ ]:
def create_sub(pred_a,pred_b,pred_c):
    submission = pd.read_csv('sample_submission.csv')
    submission['prediction'] = np.concatenate([pred_a,pred_b,pred_c])
    submission.loc[submission['prediction'] < 0, 'prediction'] = 0
    return submission

cat_pred = create_sub(pred_a,pred_b,pred_c)

### 2.Autogluon

## Part 3: Blend predictions and submit